# Trading Bot Quick Checks

This notebook provides quick sanity checks for our Bollinger Bands + MACD trading strategy.

**Disclaimer:** This is for research and educational purposes only. Not financial advice.

In [ ]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from src.utils.config import AppConfig
from src.data.ohlcv_downloader import fetch_ohlcv
from src.indicators.factory import add_indicators
from src.strategy.bb_macd_strategy import build_signals
from src.backtest.engine import run_backtest

plt.style.use('default')
plt.rcParams['figure.figsize'] = [12, 8]

## Configuration

In [ ]:
config = AppConfig()

print(f"Exchange: {config.exchange}")
print(f"Symbol: {config.symbol}")
print(f"Timeframe: {config.timeframe}")
print(f"Candle Limit: {config.candle_limit}")
print(f"\nStrategy Config:")
print(f"  Bollinger Bands: length={config.strategy.bollinger.length}, std={config.strategy.bollinger.std}")
print(f"  MACD: fast={config.strategy.macd.fast}, slow={config.strategy.macd.slow}, signal={config.strategy.macd.signal}")
print(f"  RSI: length={config.strategy.rsi.length}, filter={config.strategy.rsi.use_filter}")

## Data Loading & Indicators

In [ ]:
# Note: This will attempt to fetch live data. 
# If it fails, you can load from existing CSV file instead.

try:
    # Fetch fresh data
    df = fetch_ohlcv(config.exchange, config.symbol, config.timeframe, limit=500)
    print(f"Fetched {len(df)} candles from {df.index[0]} to {df.index[-1]}")
except Exception as e:
    print(f"Failed to fetch live data: {e}")
    print("Using mock data instead...")
    
    # Create mock data for demonstration
    dates = pd.date_range('2023-01-01', periods=500, freq='5T')
    np.random.seed(42)
    
    base_price = 45000
    price_changes = np.random.normal(0, 0.002, 500)
    prices = [base_price]
    
    for change in price_changes[1:]:
        prices.append(prices[-1] * (1 + change))
    
    df = pd.DataFrame({
        'open': prices,
        'high': [p * (1 + abs(np.random.normal(0, 0.001))) for p in prices],
        'low': [p * (1 - abs(np.random.normal(0, 0.001))) for p in prices],
        'close': prices,
        'volume': np.random.uniform(100, 1000, 500)
    }, index=dates)
    
    print(f"Created mock data: {len(df)} candles")

print(f"\nData shape: {df.shape}")
print(f"Date range: {df.index[0]} to {df.index[-1]}")
print(f"Price range: ${df['close'].min():.2f} - ${df['close'].max():.2f}")

In [ ]:
# Add technical indicators
df_with_indicators = add_indicators(df, config.strategy)

print(f"Data with indicators: {df_with_indicators.shape}")
print(f"\nColumns: {list(df_with_indicators.columns)}")
print(f"\nLast 5 rows:")
print(df_with_indicators.tail())

## Technical Analysis Charts

In [ ]:
# Price with Bollinger Bands
fig, ax = plt.subplots(figsize=(14, 8))

# Plot price and bands
ax.plot(df_with_indicators.index, df_with_indicators['close'], 
        label='Close Price', color='black', linewidth=1)
ax.plot(df_with_indicators.index, df_with_indicators['BBU'], 
        label='BB Upper', color='red', alpha=0.7, linewidth=1)
ax.plot(df_with_indicators.index, df_with_indicators['BBM'], 
        label='BB Middle', color='orange', alpha=0.7, linewidth=1)
ax.plot(df_with_indicators.index, df_with_indicators['BBL'], 
        label='BB Lower', color='green', alpha=0.7, linewidth=1)

# Fill between bands
ax.fill_between(df_with_indicators.index, 
                df_with_indicators['BBL'], df_with_indicators['BBU'], 
                alpha=0.1, color='blue')

ax.set_title('Price Action with Bollinger Bands', fontsize=16, fontweight='bold')
ax.set_ylabel('Price', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
bb_width = ((df_with_indicators['BBU'] - df_with_indicators['BBL']) / df_with_indicators['BBM'] * 100)
print(f"Bollinger Band Width - Mean: {bb_width.mean():.2f}%, Std: {bb_width.std():.2f}%")

In [ ]:
# MACD Chart
fig, ax = plt.subplots(figsize=(14, 6))

# Plot MACD components
ax.plot(df_with_indicators.index, df_with_indicators['MACD'], 
        label='MACD Line', color='blue', linewidth=1.5)
ax.plot(df_with_indicators.index, df_with_indicators['MACD_SIGNAL'], 
        label='Signal Line', color='red', linewidth=1.5)
ax.bar(df_with_indicators.index, df_with_indicators['MACD_HIST'], 
       label='Histogram', alpha=0.3, color='gray', width=0.8)

# Zero line
ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)

ax.set_title('MACD Indicator', fontsize=16, fontweight='bold')
ax.set_ylabel('MACD', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# MACD statistics
macd_crossovers = ((df_with_indicators['MACD'].shift(1) <= df_with_indicators['MACD_SIGNAL'].shift(1)) & 
                  (df_with_indicators['MACD'] > df_with_indicators['MACD_SIGNAL'])).sum()
print(f"MACD Bullish Crossovers: {macd_crossovers}")

macd_crossovers_bear = ((df_with_indicators['MACD'].shift(1) >= df_with_indicators['MACD_SIGNAL'].shift(1)) & 
                       (df_with_indicators['MACD'] < df_with_indicators['MACD_SIGNAL'])).sum()
print(f"MACD Bearish Crossovers: {macd_crossovers_bear}")

In [ ]:
# RSI Chart
fig, ax = plt.subplots(figsize=(14, 6))

# Plot RSI
ax.plot(df_with_indicators.index, df_with_indicators['RSI'], 
        label='RSI', color='purple', linewidth=1.5)

# Overbought/Oversold levels
ax.axhline(y=70, color='red', linestyle='--', alpha=0.7, label='Overbought (70)')
ax.axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Oversold (30)')
ax.axhline(y=50, color='gray', linestyle='-', alpha=0.5, label='Midline')

# Fill overbought/oversold areas
ax.fill_between(df_with_indicators.index, 70, 100, alpha=0.1, color='red')
ax.fill_between(df_with_indicators.index, 0, 30, alpha=0.1, color='green')

ax.set_title('RSI (Relative Strength Index)', fontsize=16, fontweight='bold')
ax.set_ylabel('RSI', fontsize=12)
ax.set_ylim(0, 100)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# RSI statistics
rsi_overbought = (df_with_indicators['RSI'] >= 70).sum()
rsi_oversold = (df_with_indicators['RSI'] <= 30).sum()
print(f"RSI Overbought periods (RSI >= 70): {rsi_overbought} ({rsi_overbought/len(df_with_indicators)*100:.1f}%)")
print(f"RSI Oversold periods (RSI <= 30): {rsi_oversold} ({rsi_oversold/len(df_with_indicators)*100:.1f}%)")
print(f"RSI Average: {df_with_indicators['RSI'].mean():.1f}")

## Strategy Signals

In [ ]:
# Generate trading signals
buy_signals, sell_signals = build_signals(df_with_indicators, config.strategy)

print(f"Total Buy Signals: {buy_signals.sum()}")
print(f"Total Sell Signals: {sell_signals.sum()}")
print(f"Signal Rate: {(buy_signals.sum() + sell_signals.sum()) / len(df_with_indicators) * 100:.2f}%")

if buy_signals.any():
    print(f"\nFirst Buy Signal: {buy_signals[buy_signals].index[0]}")
    print(f"Last Buy Signal: {buy_signals[buy_signals].index[-1]}")

if sell_signals.any():
    print(f"\nFirst Sell Signal: {sell_signals[sell_signals].index[0]}")
    print(f"Last Sell Signal: {sell_signals[sell_signals].index[-1]}")

In [ ]:
# Chart with signals
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12))

# Price chart with signals
ax1.plot(df_with_indicators.index, df_with_indicators['close'], 
         color='black', linewidth=1, label='Close Price')
ax1.plot(df_with_indicators.index, df_with_indicators['BBU'], 
         color='red', alpha=0.7, linewidth=1, label='BB Upper')
ax1.plot(df_with_indicators.index, df_with_indicators['BBL'], 
         color='green', alpha=0.7, linewidth=1, label='BB Lower')

# Plot buy/sell signals
if buy_signals.any():
    buy_points = df_with_indicators[buy_signals]['close']
    ax1.scatter(buy_points.index, buy_points.values, 
               marker='^', color='green', s=100, label=f'Buy Signals ({len(buy_points)})', zorder=5)

if sell_signals.any():
    sell_points = df_with_indicators[sell_signals]['close']
    ax1.scatter(sell_points.index, sell_points.values, 
               marker='v', color='red', s=100, label=f'Sell Signals ({len(sell_points)})', zorder=5)

ax1.set_title('Trading Signals on Price Chart', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# MACD with signals
ax2.plot(df_with_indicators.index, df_with_indicators['MACD'], 
         color='blue', linewidth=1.5, label='MACD')
ax2.plot(df_with_indicators.index, df_with_indicators['MACD_SIGNAL'], 
         color='red', linewidth=1.5, label='Signal')
ax2.bar(df_with_indicators.index, df_with_indicators['MACD_HIST'], 
        alpha=0.3, color='gray', width=0.8)
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)

# Mark crossover points on MACD
if buy_signals.any():
    buy_macd = df_with_indicators[buy_signals]['MACD']
    ax2.scatter(buy_macd.index, buy_macd.values, 
               marker='^', color='green', s=100, zorder=5)

if sell_signals.any():
    sell_macd = df_with_indicators[sell_signals]['MACD']
    ax2.scatter(sell_macd.index, sell_macd.values, 
               marker='v', color='red', s=100, zorder=5)

ax2.set_title('MACD with Signal Points', fontsize=16, fontweight='bold')
ax2.set_ylabel('MACD', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Quick Backtest

In [ ]:
# Run backtest
if buy_signals.any() or sell_signals.any():
    pf = run_backtest(df_with_indicators, buy_signals, sell_signals, config.strategy)
    
    # Get basic stats
    stats = pf.stats()
    
    print("=" * 50)
    print("BACKTEST RESULTS")
    print("=" * 50)
    print(f"Start Value: ${stats['Start Value']:,.2f}")
    print(f"End Value: ${stats['End Value']:,.2f}")
    print(f"Total Return: {(stats['End Value'] / stats['Start Value'] - 1) * 100:.2f}%")
    print(f"Total Trades: {stats['Total Trades']}")
    
    if stats['Total Trades'] > 0:
        print(f"Win Rate: {stats['Win Rate [%]']:.1f}%")
        print(f"Profit Factor: {stats['Profit Factor']:.2f}")
        print(f"Max Drawdown: {stats['Max Drawdown [%]']:.2f}%")
        print(f"Avg Trade: {stats['Avg Trade [%]']:.2f}%")
        print(f"Best Trade: {stats['Best Trade [%]']:.2f}%")
        print(f"Worst Trade: {stats['Worst Trade [%]']:.2f}%")
    
    print("=" * 50)
    
else:
    print("No signals generated - cannot run backtest")
    pf = None

In [ ]:
# Plot equity curve if backtest was run
if pf is not None and stats['Total Trades'] > 0:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
    
    # Portfolio value
    portfolio_value = pf.value()
    ax1.plot(portfolio_value.index, portfolio_value.values, 
             linewidth=2, color='blue', label='Portfolio Value')
    ax1.set_title('Portfolio Equity Curve', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Portfolio Value ($)', fontsize=12)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Drawdown
    drawdown = pf.drawdowns.drawdown * 100
    ax2.fill_between(drawdown.index, drawdown.values, 0, 
                     color='red', alpha=0.3, label='Drawdown')
    ax2.set_title('Drawdown (%)', fontsize=16, fontweight='bold')
    ax2.set_ylabel('Drawdown (%)', fontsize=12)
    ax2.set_xlabel('Date', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Show some trade details if available
    try:
        trades = pf.trades.records_readable
        if not trades.empty:
            print(f"\nSample trades (first 5):")
            print(trades.head().to_string(index=False))
    except:
        print("Could not retrieve trade records")
        
elif pf is not None:
    print("No trades executed - portfolio remained unchanged")
else:
    print("No backtest to display")

## Summary

This notebook demonstrated:

1. **Data Loading**: Fetched OHLCV data and computed technical indicators
2. **Technical Analysis**: Visualized Bollinger Bands, MACD, and RSI
3. **Signal Generation**: Applied the BB-MACD strategy rules
4. **Backtesting**: Ran a quick performance test

### Next Steps
- Try different timeframes or symbols
- Adjust strategy parameters in `config/strategy.yaml`
- Run full pipeline with CLI: `python -m src.cli pipeline`
- Review detailed backtest reports in `reports/` directory

### Important Notes
- This is for research and educational purposes only
- Past performance does not guarantee future results
- Always validate strategies with out-of-sample data
- Consider transaction costs, slippage, and market conditions